In [11]:
import json
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def json_to_xml(json_file, output_dir):
    with open(json_file) as f:
        data = json.load(f)
    
    for i in data['images']:
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'folder').text = 'JPEGImages'
        # Check if extension is .png
        if i['file_name'].endswith('.png'):
            # Replace .png with .jpg
            ET.SubElement(annotation, 'filename').text = i['file_name'].replace('.png', '.jpg')
            #ET.SubElement(annotation, 'filename').text = i['file_name']
        source = ET.SubElement(annotation, 'source')
        ET.SubElement(source, 'database').text = 'Unknown'
        size = ET.SubElement(annotation, 'size')
        ET.SubElement(size, 'width').text = str(i['width'])
        ET.SubElement(size, 'height').text = str(i['height'])
        ET.SubElement(size, 'depth').text = '3'
        ET.SubElement(annotation, 'segmented').text = '0'

        for a in data['annotations']:
            if a['image_id'] == i['id']:
                for c in data['categories']:
                    if a['category_id'] == c['id']:
                        object = ET.SubElement(annotation, 'object')
                        ET.SubElement(object, 'name').text = c['name']
                        ET.SubElement(object, 'pose').text = 'Unspecified'
                        ET.SubElement(object, 'truncated').text = '0'
                        ET.SubElement(object, 'difficult').text = '0'
                        bndbox = ET.SubElement(object, 'bndbox')
                        ET.SubElement(bndbox, 'xmin').text = str(int(a['bbox'][0]))
                        ET.SubElement(bndbox, 'ymin').text = str(int(a['bbox'][1]))
                        ET.SubElement(bndbox, 'xmax').text = str(int(a['bbox'][0] + a['bbox'][2]))
                        ET.SubElement(bndbox, 'ymax').text = str(int(a['bbox'][1] + a['bbox'][3]))

        xml = prettify(annotation)
        with open(os.path.join(output_dir, i['file_name'].replace('.png', '.xml')), 'w') as f:
            f.write(xml)


In [12]:
# Usage
json_to_xml('/home/enzo.viacava/annotations/instances_default_ev.json', '/home/enzo.viacava/pascal_voc_ev')